https://docs.daft.ai/en/stable/quickstart/#read-from-a-data-source

In [5]:
import daft

df = daft.from_pydict({
    "A":[1,2,3,4],
    "B":[1.5,2.5,3.5,4.5],
    "C":[True,True,False,False],
    "D":[None,None,None,None],
})

df

AInt64,BFloat64,CBool,DNull
1,1.5,true,None
2,2.5,true,None
3,3.5,false,None
4,4.5,false,None


In [11]:
daft.set_planning_config(default_io_config=daft.io.IOConfig(s3=daft.io.S3Config(anonymous=True)))

df = daft.read_parquet("s3://daft-public-data/tutorials/10-min/sample-data-dog-owners-partitioned.pq/**")
df

first_nameString,last_nameString,ageInt64,DoBDate,countryString,has_dogBool


In [13]:
df.select("first_name", "has_dog").show()

first_nameString,has_dogBool
Ernesto,true
James,true
Wolfgang,None
Shandra,true
Zaya,true


In [14]:
df.where(daft.col("age") >= 40).show()

first_nameString,last_nameString,ageInt64,DoBDate,countryString,has_dogBool
James,Jale,62,1962-03-24,Canada,true
Shandra,Shamas,57,1967-01-02,United Kingdom,true
Zaya,Zaphora,40,1984-04-07,United Kingdom,true


In [15]:
df.show(3)

first_nameString,last_nameString,ageInt64,DoBDate,countryString,has_dogBool
Shandra,Shamas,57,1967-01-02,United Kingdom,true
Zaya,Zaphora,40,1984-04-07,United Kingdom,true
Ernesto,Evergreen,34,1990-04-03,Canada,true


In [18]:
df.limit(3)

first_nameString,last_nameString,ageInt64,DoBDate,countryString,has_dogBool


In [17]:
df.limit(3).show()

first_nameString,last_nameString,ageInt64,DoBDate,countryString,has_dogBool
Wolfgang,Winter,23,2001-02-12,Germany,None
Ernesto,Evergreen,34,1990-04-03,Canada,true
James,Jale,62,1962-03-24,Canada,true


In [21]:
df.exclude("DoB").show()

first_nameString,last_nameString,ageInt64,countryString,has_dogBool
Ernesto,Evergreen,34,Canada,true
James,Jale,62,Canada,true
Shandra,Shamas,57,United Kingdom,true
Zaya,Zaphora,40,United Kingdom,true
Wolfgang,Winter,23,Germany,None


In [23]:
df = df.with_column("full_name", daft.col("first_name") + " " + daft.col("last_name"))
df.select("full_name", "age", "country", "has_dog").show()

full_nameString,ageInt64,countryString,has_dogBool
Ernesto Evergreen,34,Canada,true
James Jale,62,Canada,true
Wolfgang Winter,23,Germany,None
Shandra Shamas,57,United Kingdom,true
Zaya Zaphora,40,United Kingdom,true


In [24]:
df.select((daft.col("first_name").alias("full_name") + " " + daft.col("last_name")), "age", "country", "has_dog").show()

full_nameString,ageInt64,countryString,has_dogBool
Ernesto Evergreen,34,Canada,true
James Jale,62,Canada,true
Wolfgang Winter,23,Germany,None
Shandra Shamas,57,United Kingdom,true
Zaya Zaphora,40,United Kingdom,true


In [26]:
df.sort(daft.col("age"), desc=False).show()

first_nameString,last_nameString,ageInt64,DoBDate,countryString,has_dogBool,full_nameString
Wolfgang,Winter,23,2001-02-12,Germany,None,Wolfgang Winter
Ernesto,Evergreen,34,1990-04-03,Canada,true,Ernesto Evergreen
Zaya,Zaphora,40,1984-04-07,United Kingdom,true,Zaya Zaphora
Shandra,Shamas,57,1967-01-02,United Kingdom,true,Shandra Shamas
James,Jale,62,1962-03-24,Canada,true,James Jale


In [27]:
grouped = df.groupby("country").agg(
    daft.col("age").mean().alias("avg_age"),
    daft.col("has_dog").count()
).show()

countryString,avg_ageFloat64,has_dogUInt64
United Kingdom,48.5,2
Canada,48,2
Germany,23,0
